#ME615 Tutorial 2

# 1-D Elliptic problem (steady-state head conduction)


The non-dimensional 1-D heat conduction equation for a fin is given as:

GDE: $\frac{d^2 \theta}{d x^2} = \Pi \theta$, where $\Pi$ is a non-dimensional parameter,

BC 1: $\theta(x=0)=1$, and

BC 2: $\left( \frac{d \theta}{d x} \right)_{x=1}=0$.

You have to solve this problem for $\theta(x)$, where $x \in [0,1]$.

Please note that you have only one non-dimensional parameter $\Pi$ in this problem. The solution to the above system depends on this parameter only.

### Exercise 1:
1. Write the discretised equations corresponding to the above system.
2. Write the above equations as a linear system $A \theta = r$, showing the matrix $A$ and RHS vector $r$.
3. Write a Python program to solve for the $\theta$ vector, for $\Pi=0.01$ and $N=10$, which corresponds to $N+1$ nodal points. You may store the full A matrix and invert it to calculate the unknown vector $\theta$.
4. Plot $\theta$ as a function of $x$. The present problem has an analytical solution that can be used to test the implementation; $\theta_{a}=\frac{\cosh(\sqrt{\pi} (1-x))}{\cosh{\sqrt{\pi}}}$. Plot the analytical solution on the same graph too.

In [ ]:
# %%timeit

import numpy as np
import matplotlib.pyplot as plt
import math

pie=0.01
N=10

h=1.0/(N)
a=1./h**2
b=1./h**2
d=-(2/h**2 + pie)

# inefficient method that stores the full matrix A
A=np.zeros((N+1,N+1))
A[0][0]=1
A[N][N]=1
A[N][N-1]=-1
for i in range(1,N):
    A[i][i]=d
    A[i][i+1]=a
    A[i][i-1]=b

r=np.zeros(N+1)
r[0]=1.

theta_num=np.dot(np.linalg.inv(A),r)
x=np.linspace(0,1,N+1)
theta_ana=np.cosh(np.sqrt(pie)*(1.-x))/np.cosh(np.sqrt(pie))

plt.plot(x,theta_num, label="numerical")
plt.plot(x,theta_ana, 'r--',label="analytical")
plt.xlabel(r"$x$")
plt.ylabel(r"$\theta$")
plt.grid()
plt.legend()
plt.show()

### Exercise 2:
1. Solve **Exercise 1** with the help of the tridiagonal matrix algorithm (TDMA).
2. Time the execution using the %%timeit magic command of Python and compare TDMA with inverse calculation method.



In [ ]:
# %%timeit   # to time the execution of the function

import numpy as np
import matplotlib.pyplot as plt
import math

pie=0.01
N=10

h=1.0/(N)

# TDMA method
# Start with \theta_1, therefore, \theta_0 denotes the first interior point from the left

# initialise the four arrays
d=np.ones(N)*(-2/h**2 - pie)
a=np.ones(N)*(1./h**2)
b=np.ones(N)*(1./h**2)
r=np.zeros(N)
theta_num=np.ones(N)
theta_num[0]=1.0

d[N-1]=1
b[N-1]=-1

r[0]=-(1./h**2)*1.0

#forward eliminitaion
for i in range(1,N):
    d[i] = d[i] - a[i-1]*(b[i]/d[i-1])
    r[i] = r[i] - r[i-1]*(b[i]/d[i-1])

#backward substitution
theta_num[N-1] = r[N-1]/d[N-1]
for i in range(N-2,0,-1):
    theta_num[i]=(r[i]-a[i]*theta_num[i+1])/d[i]

x=np.linspace(0,1,N)
theta_ana=np.cosh(np.sqrt(pie)*(1.-x))/np.cosh(np.sqrt(pie))

plt.plot(x,theta_num, label="numerical")
plt.plot(x,theta_ana, 'r--',label="analytical")
plt.xlabel(r"$x$")
plt.ylabel(r"$\theta$")
plt.grid()
plt.legend()
plt.show()

### Exercise 3
Plot $\theta$ vs $x$ for different values of $N$ (=10, 50, 100, 1000). Also plot the analytical solution on the same graph and compare. What happens as you increase the value of $N$, and why? Is it desirable to choose a very large value for $N$? What is the optimum $N$ value. What can you say about *grid independence* from the graph?

There is a trade-off between the solution accuracy and the computation time; however, beyond a certain value of $N$ the solution does note change much -- the solution is said to become *grid independent* beyond this value of $N$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

pie=0.01
for N in [10, 50, 100, 1000]:
    h=1.0/(N)

    # initialise the four arrays
    d=np.ones(N)*(-2/h**2 - pie)
    a=np.ones(N)*(1./h**2)
    b=np.ones(N)*(1./h**2)
    r=np.zeros(N)
    theta_num=np.ones(N)
    theta_num[0]=1.0

    d[N-1]=1
    b[N-1]=-1

    r[0]=-(1./h**2)*1.0

    #forward eliminitaion
    for i in range(1,N):
        d[i] = d[i] - a[i-1]*(b[i]/d[i-1])
        r[i] = r[i] - r[i-1]*(b[i]/d[i-1])

    #backward substitution
    theta_num[N-1] = r[N-1]/d[N-1]
    for i in range(N-2,0,-1):
        theta_num[i]=(r[i]-a[i]*theta_num[i+1])/d[i]

    x=np.linspace(0,1,N)
    plt.plot(x,theta_num, label="N="+str(N))

N=20
x=np.linspace(0,1,N)
theta_ana=np.cosh(np.sqrt(pie)*(1.-x))/np.cosh(np.sqrt(pie))
plt.plot(x,theta_ana, 'b*', label="analytical")
plt.xlabel(r"$x$")
plt.ylabel(r"$\theta_{num}$")
plt.grid()
plt.legend()
plt.show()

### Exercise 4
Plot the error in numerical solution (at $x=0.5$) versus $\Delta x$ on a log-log graph and find the order of convergence.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

pie=0.0001
N_list=[10, 50, 100, 1000]
error=[]
for N in N_list:
    h=1.0/(N)

    # initialise the four arrays
    d=np.ones(N)*(-2/h**2 - pie)
    a=np.ones(N)*(1./h**2)
    b=np.ones(N)*(1./h**2)
    r=np.zeros(N)
    theta_num=np.ones(N)
    theta_num[0]=1.0

    d[N-1]=-1
    b[N-1]=1

    r[0]=-(1./h**2)*1.0

    #forward eliminitaion
    for i in range(1,N):
        d[i] = d[i] - a[i-1]*(b[i]/d[i-1])
        r[i] = r[i] - r[i-1]*(b[i]/d[i-1])

    #backward substitution
    theta_num[N-1] = r[N-1]/d[N-1]
    for i in range(N-2,0,-1):
        theta_num[i]=(r[i]-a[i]*theta_num[i+1])/d[i]

    x=np.linspace(0,1,N)
    theta_ana=np.cosh(np.sqrt(pie)*(1.-x))/np.cosh(np.sqrt(pie))
    err=theta_num[int(N/2)]-theta_ana[int(N/2)]
    error.append(err)

plt.plot(np.log10(1/np.array(N_list)),np.log10(np.abs(error)))
plt.xlabel(r"$\Delta x$")
plt.ylabel(r"error")
plt.grid()
plt.show()